<a href="https://colab.research.google.com/github/AleemKhan12/AleemKhan12/blob/calendar_data_algorithm/Attendance_Tracking_Calendar_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Attendance Tracking by using Calendar and Audit Log Data

#Revision notes V0.11: download new audit log data, change time zone code, participant identifier to actor

'''
ISSUES
1. Reading google sheet for Audit data returning wrong values
2. Test time conversion accuracy. Punctuality returning wrong values
3. Finalize naming convention for student/mentee

'''


'\nISSUES\n1. Reading google sheet for Audit data returning wrong values\n2. Test time conversion accuracy. Punctuality returning wrong values\n3. Finalize naming convention for student/mentee\n\n'

# CONNECTING TO GOOGLE DRIVE

In [1]:
#connect colab to drive

from google.colab import drive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import gspread
drive.mount('/content/gdrive')
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


Mounted at /content/gdrive


In [2]:
#Imports
import pandas as pd
from datetime import date, timedelta
import pytz
from numpy.core.numeric import NaN
import datetime
import math



In [ ]:
pytz.all_timezones

#IMPORTANT GLOBAL VARIABLES

In [3]:
mentor_email = 'mentor@villagebookbuilders.org' #Mentor program email address
oracle_data       = 0                           #Data read from Oracle Database SQL Query
google_data       = 0                           #Data read from Google Audit Log
active_sessions   = 0                           #Active sessions recorded in Oracle db based on end date
total_mentors     = 0                           #List of all mentors names in oracle db
oracle_mentor_emails = []                       #list of all mentor emails in oracle db
active_mentors    = 0                           #Mentor names that joined at least one session 
active_mentor_emails = []                       #List of mentor emails that joined at least one session
mentor_programs   = []                          #Names of all mentor programs stored in oracle database
mentors_not_registered = []                     #Names of all active mentors not logged in oracle database
time_zone = 'America/Port_of_Spain'                         #google_data['Date'][1][len(google_data['Date'][1])-3:] #TODO: change this hard coded time zone
start = 0                                       #start date
end = 0                                         #end date
mentor_identifier = 'Mentor'                    #Mentor identifier
mentee_identifier = 'Student'                   #Mentee identifier
cal_id_len = 26                                 #usable calendar event id string length
google_resource_identifier = '@resource.calendar.google.com'
fire_flies_email = 'fred@fireflies.ai'
program_director_name = ['adeiso ghana', 'ahero kenya', 'biwi, malawi vbb mentor program', 'charity centre, zambia vbb mentor program', 'chebosi kenya', 'kadzakalowa malawi', 'maragoli kenya', 'mukono uganda', 'vima ghana'] #Actor names of program director accounts

# READING DATA SOURCES

In [7]:
#Read Data Sources as pandas data frames

#Calendar Data
sheet_url = 'https://docs.google.com/spreadsheets/d/1KzR1ok2TEwRQbta534v-DvpSvtp-LlEofL8hORqBjWI/edit#gid=0'
calendar_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#calendar_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Calendar Data Export/Calendar Export UTC - Sheet1.csv' 
calendar_data = pd.read_csv(calendar_path, on_bad_lines='skip')

#Google Audit Log Data
# sheet_url = 'https://docs.google.com/spreadsheets/d/1O81Lc9neAFY5KatvApTAJJwyUXKEldEJe8kdnSM0YEI/edit#gid=0'
# google_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
google_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Google Audit Log/3 6 2022 - 3 12 2022/3_6_2022 - 3_12_2022.csv'
google_data = pd.read_csv(google_path, on_bad_lines='skip')


#Mentee Computer Emails
sheet_url = 'https://docs.google.com/spreadsheets/d/1JcJ99pX-pjMTCkFrLwjpaCJ_1FcJ2xZ7UqgBJHGFl9I/edit#gid=925063253'
mentee_email_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#mentee_email_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/mentee email/Copy of Village Library Information - computer email.csv'
mentee_computer_emails = pd.read_csv(mentee_email_path, on_bad_lines='skip')

#Corporate Parners
sheet_url = 'https://docs.google.com/spreadsheets/d/1K9FtfVv-lzU0eQE4V27Qdb3zuj2t9d-WOxs2ZAA04kw/edit#gid=0'
corporate_partners_path = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
#corporate_partners_path = '/content/gdrive/MyDrive/Projects/Mentor_Dashboard_V2/3. Execution/SRC Code/Data/Corporate Organizations/Corporate Partners.csv'
corporate_partners = pd.read_csv(corporate_partners_path, on_bad_lines='skip')



In [9]:
print(calendar_data.shape, google_data.shape, mentee_computer_emails.shape, corporate_partners.shape)

(1477, 11) (480, 31) (132, 2) (3, 2)



# CLEANING DATA

In [12]:

#Calendar Data
calendar_data = calendar_data.dropna() #remove all empty sessions
calendar_data = calendar_data.drop_duplicates()
calendar_data = calendar_data[(~calendar_data['Participant Identifier'].str.contains(google_resource_identifier)) & (calendar_data['Participant Identifier'] != fire_flies_email)]

#Google Data
google_data = google_data.dropna(subset=['Date', 'Meeting code', 'Actor', 'Organizer email', 'Duration (seconds)', 'Actor name', 'Calendar event ID']) #clean google data
google_data = google_data.drop_duplicates()
google_data = google_data[google_data['Duration (seconds)'] > 0] #drop all calls with 0 duration
google_data = google_data[google_data['Organizer email'] == mentor_email] #only consider mentor sessions
google_data = google_data[google_data['Actor name'] != 'Mentor Program'] #drop all entries for mentor program

#Mentee Email data
mentee_computer_emails = mentee_computer_emails.dropna()

#Corporate Parner data
corporate_partners = corporate_partners.dropna()

In [15]:
print(calendar_data.shape, google_data.shape)

(1074, 11) (404, 31)


In [16]:
#DATE CONVERSIONS
#Only run this once. Running more times will cause an error. To fix re-read data sources
def conv_to_datetime(a):
  utc_timezone = pytz.timezone('UTC')
  if isinstance(a, datetime.date):
    return a
  try:
    #Convert calendar export dates
    date = datetime.datetime.strptime(a, '%m/%d/%Y %H:%M:%S')
    date = date.replace(tzinfo = pytz.timezone('UTC')) 
    return date
  except:
    try:
      #Convert Audit log dates 
      date = datetime.datetime.strptime(a, '%Y-%m-%dT%H:%M:%SZ')
      date = date.replace(tzinfo = pytz.timezone('UTC')) 
      return  date
    except:
      date = datetime.datetime.strptime('', '')
      date = date.replace(tzinfo = utc_timezone)
      return date



In [ ]:
google_data.head()

In [19]:
#converting calendar dates to datetime objs
calendar_data['Event Start UTC'] = calendar_data['Event Start UTC'].apply(conv_to_datetime)
calendar_data['Event End UTC'] = calendar_data['Event End UTC'].apply(conv_to_datetime)



In [ ]:
#Getting usable substring from Calendar Event ID
calendar_data['Calendar event ID'] = calendar_data['Calendar event ID'].apply(lambda x: x[:cal_id_len])

In [21]:
#Convert google dates to datetime UTC
#google_data['Date'] = google_data['Date'].apply(lambda x: x[:-4]) #remove time zone
google_data['Date'] = google_data['Date'].apply(conv_to_datetime)

In [ ]:
google_data.head()

In [23]:
#Create new column with just dates
google_data['String_Date'] = google_data['Date'].apply(lambda x: x.strftime("%m/%d/%Y"))
calendar_data['String_Date'] = calendar_data['Event Start UTC'].apply(lambda x: x.strftime("%m/%d/%Y"))

In [25]:
#lower participant emails for accurate searches
google_data['Actor'] = google_data['Actor'].apply(lambda x: x.lower())
calendar_data['Participant Identifier'] = calendar_data['Participant Identifier'].apply(lambda x: x.lower())

In [ ]:
#Drop duplicate sessions caused by people signing in from various devices at the same time
#google_data = google_data.drop_duplicates(subset = ['Participant Identifier', 'Calendar event ID', 'String_Date', 'IP address'])

In [26]:
#list of mentee computer emails
mentee_computer_emails_list = mentee_computer_emails['COMPUTER_EMAIL'].tolist()

#DATE TIME TESTS

In [ ]:
google_data.head()

,Date,Event,Event Description,Meeting code,Participant Identifier,Participant outside organization,Client type,Organizer email,Product type,Duration,...,Video Send Packet Loss Max,Video Send Packet Loss Mean,Video Send Duration,Video Send Short Side Median,Action reason,Action description,Target display names,Target,Target phone number,String_Date
0,2022-03-05 18:26:28+00:00,Endpoint left,The endpoint left a video meeting,UPZYGGFBSJ,nila.muthusamy@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,15493.0,...,0.0,0.0,10,180.0,NaN,NaN,NaN,NaN,NaN,03/05/2022
6,2022-03-05 16:27:16+00:00,Endpoint left,The endpoint left a video meeting,VXRUJYWHEB,mpmukono@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,4702.0,...,2.0,0.0,4707,180.0,NaN,NaN,NaN,NaN,NaN,03/05/2022
7,2022-03-05 16:27:10+00:00,Endpoint left,The endpoint left a video meeting,VXRUJYWHEB,kiran.sharma@villagementors.org,No,Web,mentor@villagebookbuilders.org,Google Meet,4704.0,...,2.0,0.0,2037,360.0,NaN,NaN,NaN,NaN,NaN,03/05/2022
11,2022-03-05 16:02:32+00:00,Endpoint left,The endpoint left a video meeting,BJMYWJTVVA,mpcharitycentre@villagebookbuilders.org,No,Android,mentor@villagebookbuilders.org,Google Meet,5274.0,...,1.0,0.0,2824,180.0,NaN,NaN,NaN,NaN,NaN,03/05/2022
12,2022-03-05 15:54:29+00:00,Endpoint left,The endpoint left a video meeting,QIUZBJEQVJ,xyz082719@gmail.com,Yes,Web,mentor@villagebookbuilders.org,Google Meet,2365.0,...,NaN,NaN,2369,360.0,NaN,NaN,NaN,NaN,NaN,03/05/2022


In [ ]:
#Today's date
today_date = datetime.datetime.today().replace(tzinfo = pytz.timezone(time_zone))

#Start and end dates for schema 1
dates_list = sorted(google_data['Date'].tolist())
start1, end1 = dates_list[0], dates_list[-1] 

#Start and end dates for schema 2
end, start = datetime.datetime(2022, 2, 27).astimezone(pytz.timezone("utc")), datetime.datetime(2022, 3, 5).astimezone(pytz.timezone("utc"))



In [ ]:
print(start1, end1)
print(start, end)

2021-09-07 11:05:25+00:00 2022-03-05 18:26:28+00:00
2022-03-05 00:00:00+00:00 2022-02-27 00:00:00+00:00


#ATTENDANCE AND DURATION TRACKING 

In [27]:
calendar_data['Attendance'] = [0]*len(calendar_data)
calendar_data['Duration'] = [0]*len(calendar_data)
calendar_data['User Type'] = [0]*len(calendar_data)
calendar_data['Organization'] = [0]*len(calendar_data)
#calendar_data['Punctuality'] = [NaN]*len(calendar_data)

In [ ]:
from os import putenv
for index, meeting in calendar_data.iterrows():
  attendance = 0
  duration = 0
  meeting_data = google_data[(google_data['Actor'] == meeting['Participant Identifier']) & (google_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (google_data['String_Date'] == meeting['String_Date'])]

  if (len(meeting_data) > 0):
    attendance = 1
    duration = sum(meeting_data['Duration (seconds)'].tolist())/60 #duration in minutes

  calendar_data['Attendance'][index] = attendance
  calendar_data['Duration'][index] = duration
  calendar_data['User Type'][index] = mentee_identifier if meeting['Participant Identifier'] in mentee_computer_emails_list else mentor_identifier
 

  #Assigning corporate partners
  partner_name = 'Public/Chapter'
  for i, partner in corporate_partners.iterrows():
    if partner['Email Identifier'] in meeting['Participant Identifier']:
      partner_name = partner['Organization']
      break;

  calendar_data['Organization'][index] = partner_name



In [32]:
calendar_data['Missed_2'] = [0]*len(calendar_data)
calendar_data['Missed_3'] = [0]*len(calendar_data)

In [ ]:
#TODO: Write to complete list of attendance data here

#FINDING MENTOR THAT MISSED LAST 2 OR 3 SESSIONS

In [ ]:
#Identify mentors that have missed 2 or 3 of their last sessions. 

In [ ]:
for index, meeting in calendar_data.iterrows():

  #Get previous meetings regarless of id
  #TODO: Read from complete list of attendance data here, also find how to return max 3 entires 
  previous_meetings = calendar_data[(calendar_data['Event Start UTC'] <= meeting['Event Start UTC']) & (calendar_data['Participant Identifier'] == meeting['Participant Identifier'])] # & (calendar_data['Calendar event ID'] == meeting['Calendar event ID'])]

  if (len(previous_meetings)>=3):
    last_3 = 0 if sum(previous_meetings['Attendance'].tolist()[-3:]) else 1
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()[-2:]) else 1
  elif (len(previous_meetings) == 2):
    last_3 = 0
    last_2 = 0 if sum(previous_meetings['Attendance'].tolist()) else 1
  else:
    last_3 = 0
    last_2 = 0
  
  #updating data frame 
  calendar_data['Missed_2'][index] = last_2
  calendar_data['Missed_3'][index] = last_3

In [34]:
calendar_data.shape

(1074, 18)

In [ ]:
#TODO: merge actor names to calendar_data based on participant identifier
# actor_names = google_data[['Participant Identifier', 'Actor name']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)
# actor_names = calendar_data[['Participant Identifier']].drop_duplicates(subset = ['Participant Identifier'])
# print(actor_names.shape)

# #Merge actor names
# result = calendar_data#pd.merge(calendar_data, actor_names, on='Participant Identifier')

# STORING ATTENDANCE DATA

In [35]:
#Create data frame and store
#TODO: write to complete list of all attendance data; over write any redundant data
file_name = 'attendance_data_calendar_data.csv'
calendar_data.to_csv(file_name, index = False)

#FINDING PRODUCTIVE MEETINGS

In [36]:
#XOR function
def xor(a, b):
  if((a and b) or (not a and not b)):
    return False
  else:
    return True

In [37]:
#Convert list to string 
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele + ", "  
    
    # return string  
    return str1

In [ ]:
#A productive meeting will be a meeting in which at least one mentor and one mentee attended

In [38]:
#Get all meetings per day 
productive_meetings  = calendar_data[['String_Date', 'Calendar event ID', 'Program Name']].drop_duplicates(subset=['String_Date', 'Calendar event ID'])
productive_meetings['Productive'] = [0]*len(productive_meetings)
productive_meetings['Alone'] = [0]*len(productive_meetings)
productive_meetings['Absent Type'] = [NaN]*len(productive_meetings)
productive_meetings['Absent Identifier'] = [NaN]*len(productive_meetings)

In [ ]:
#For each meeting in each day see if the meeting was productive

for index, meeting in productive_meetings.iterrows():
  
  mentor_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentor_identifier)]['Attendance'].tolist())
  mentee_attendance = sum(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentee_identifier)]['Attendance'].tolist())

  #if session was productive
  if(mentor_attendance and mentee_attendance):
    productive_meetings['Productive'][index] = 1
    productive_meetings['Absent Type'][index] = NaN

  #If only one particpant showed
  elif(xor(mentor_attendance, mentee_attendance)):
    #if only one participant joined, record as Alone Session
    productive_meetings['Alone'][index] = 1      

    #get absentee type
    if mentee_attendance:
      productive_meetings['Absent Type'][index] = mentor_identifier
      productive_meetings['Absent Identifier'][index] = listToString(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentor_identifier)]['Participant Identifier'].tolist())
    else:
      if mentor_attendance:
        productive_meetings['Absent Type'][index] = mentee_identifier
        productive_meetings['Absent Identifier'][index] = listToString(calendar_data[(calendar_data['String_Date'] == meeting['String_Date']) & (calendar_data['Calendar event ID'].str.contains(meeting['Calendar event ID'])) & (calendar_data['User Type'] == mentee_identifier)]['Participant Identifier'].tolist())

  #if no one showed
  if(not productive_meetings['Productive'][index] and not productive_meetings['Alone'][index]):
    productive_meetings['Absent Type'][index] = NaN

      
      #productive_meetings['Absent Type'][index] = (mentor_identifier if mentee_attendance else (mentee_identifier if mentor_attendance else NaN))
    

# STORING PRODUCTIVE MEETINGS

In [40]:
#Create data frame and store
#TODO: write to complete list of all attendance data; over write any redundant data
file_name = 'productive_meetings.csv'
productive_meetings.to_csv(file_name, index = False)